# Text Mining Group Project
## RU-EN Corpus

##### TOC for Implemented Metrics.

---

In [1]:
# Necessary Installs.
#!pip install rouge

In [2]:
# Imports
import pandas as pd
import numpy as np
from collections import Counter
from rouge import Rouge
import string
from nltk.translate import chrf_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import StandardScaler

Load Dataset

In [3]:
df1 = pd.read_csv("scores.csv")

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17980 entries, 0 to 17979
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   source       17980 non-null  object 
 1   reference    17980 non-null  object 
 2   translation  17980 non-null  object 
 3   z-score      17980 non-null  float64
 4   avg-score    17980 non-null  float64
 5   annotators   17980 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 842.9+ KB


In [5]:
df1.head()

,source,reference,translation,z-score,avg-score,annotators
0,В этом году крымчане получат уведомление на оп...,This year the Crimeans will receive a notice f...,"This year, residents of Crimea will receive a ...",0.878043,92.0,1
1,Энергетические компании находятся под давление...,Energy companies are under pressure due to low...,Energy companies are under pressure from lower...,0.511473,81.5,2
2,В бархатный сезон покупают туры в основном оди...,"In the velvet season, tours are mostly single ...","In the autumn season, tours are mainly purchas...",0.947866,94.0,1
3,Возле него на всякий случай стоит познавательн...,"Near him, just in case, there is a cognitive t...","Beside it, for good measure, there is an infor...",1.052601,97.0,1
4,Действительно ли Эфиопия находится на грани ра...,Is Ethiopia on the verge of schism?,Is Ethiopia about to crack?,0.738397,88.0,1


---
### PreProcessing

In [6]:
# Check for empty or sparse reference / translation, and drop them.
for column in ["source","reference","translation"]:
    print(column)
    bad_idx = [idx for idx in np.where(df1[column].str.len()<=2)[0]
    print(f"Bad idx: {bad_idx}")
    df1 = df1.drop(index=bad_idx)

source
False
reference
True
translation
False


In [7]:
# Create two dataframes. One is unprocessed, the other is preprocessed to remove punctuation and be lowercased.
# (Future note: maybe also remove stopwords?)
df_u = df1.copy()
df_p = df1.copy()

for x in ["source","reference","translation"]:
    # lowercase.
    df_p[x] = df1[x].str.lower()
    # Remove punct.
    df_p[x] = df1[x].map(lambda s: s.translate(str.maketrans("","",string.punctuation))
                           .lower()
                          )
    
df_dict = {"df_u":df_u,"df_p":df_p}

In [8]:
## Initialize a scaler for later.
scaler = StandardScaler()

---
### Metrics

--- 
Bleu Metric

In [9]:
def my_first_BLEU(reference,translation):
    """
    Expects lists of strings for both reference and translation.
    Returns the score 
    """
    
    # Let word be every unique word in the translation.
    # Can be done by setting up a Counter object.
    t_c = Counter(word_tokenize(translation))
    words = sorted(t_c)
    
    refs_c = Counter(word_tokenize(reference))
    
    # Let Covered be the minimum amt of times a word appears in the reference, compared to R(w).
    # Let D(word) be how many times a unique word appears in the candidate translation.
    # Let R(word) be the largest numer of times the word appears in any one reference.
    
    covered = 0
    
    for word in words:
        covered += min(t_c[word],refs_c[word])

    

    # Let total be the number of words in translation.
    total = sum(t_c.values())
    
    BLEU_score = covered / total
    
    return BLEU_score


In [10]:
bleu_scores_list = ["basic bleu"]

for df in list(df_dict.values()):
    for key in bleu_scores_list:
        # Apply the function to get a column of the scores.
        df[(key+" score")] = df.apply(lambda row: my_first_BLEU(row["reference"],row["translation"]),axis=1)
        # Also add a z score column.
        df[(key+ "_zscore")] = scaler.fit_transform(df[(key+" score")].to_numpy().reshape(-1,1)).flatten()

---
Rouge metric as described in

https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460

And
https://pypi.org/project/rouge-metric/

In [11]:
rouge = Rouge()

In [12]:
# Example Cell
model_out = df1["translation"][0]
reference = df1["reference"][0]
# The get scores method returns three metrics, F1 score, p precision and recall r.
# For each unigram,bigram and Longest sequence.
rouge.get_scores(model_out,reference)

[{'rouge-1': {'f': 0.5185185135253774, 'p': 0.5384615384615384, 'r': 0.5},
  'rouge-2': {'f': 0.23999999500800012, 'p': 0.25, 'r': 0.23076923076923078},
  'rouge-l': {'f': 0.4615384565384616,
   'p': 0.46153846153846156,
   'r': 0.46153846153846156}}]

In [13]:
# Get scores.
# For every df considered;
for df in list(df_dict.values()):
    # For the entire model, model_out and reference need to be lists of strings.
    model_out = df["translation"].to_list()
    reference = df["reference"].to_list()
    rouge_scores = rouge.get_scores(model_out,reference)
    # For each of the scores calculated, output a new column in the df with the f1 scores.
    for key in rouge_scores[0].keys():
        df[(key+" score")] = pd.Series([score[key]["f"] for score in rouge_scores])
        # Also add a z score column.
        df[(key+ "_zscore")] = scaler.fit_transform(df[(key+" score")].to_numpy().reshape(-1,1)).flatten()

---
#### chrF metric

Check the paper here: https://www.aclweb.org/anthology/W15-3049.pdf

The general formula for the CHRF score is:

`CHRFBeta = (1 + Beta**2) * ((chrP * chrR) / (Beta**2*chrP + chrR))`

where:
* chrP is the percentage of n-grams in the hypothesis which have a counterpart in the reference.
* chrR is the percentage of character n-grams in the reference which are also present in the hypothesis.
* Beta is a parameter which assigns beta times more importance to recall than to precision (if beta == 1, they have the same importance).

In [14]:
# I was surprised, but this works exactly like it's intended. Makes a new column with the chrF score for each row of the df.
# The default n-gram values are min == 1, max == 6. 
# The default beta is 3.

# Moreover, it is worthwhile to mention chrf uses its own tokenization with whitespaces.
# All parameters to test chrf scores with. feel free to play around with this and test out different combinations.
# Note: this takes a few minutes to run.
min_len = [1,2]
max_len = [6,10]
beta = [1,3]

for df in list(df_dict.values()):
    chrf_scores=[]
    for min_l in min_len:
        for max_l in max_len:
            for b in beta:
                append_str = "chrf_b" + str(b) + "_n" + str(min_l) + str(max_l)
                chrf_scores.append(append_str)
                df[append_str] = df.apply(lambda row: chrf_score.sentence_chrf(row["reference"],row["translation"],min_len=min_l,max_len=max_l,beta=b),axis=1)
                # Also add a z score column.
                df[(append_str+ "_zscore")] = scaler.fit_transform(df[append_str].to_numpy().reshape(-1,1)).flatten()

df_p.loc[:,chrf_scores]

,chrf_b1_n16,chrf_b3_n16,chrf_b1_n110,chrf_b3_n110,chrf_b1_n26,chrf_b3_n26,chrf_b1_n210,chrf_b3_n210
0,0.620559,0.612515,0.485457,0.479028,0.560544,0.553200,0.437104,0.431244
1,0.600948,0.551251,0.484572,0.444147,0.550094,0.504420,0.443389,0.406229
2,0.443000,0.490761,0.320972,0.356228,0.374119,0.415000,0.269147,0.299191
3,0.642672,0.676944,0.556823,0.586909,0.590986,0.622665,0.518570,0.546750
4,0.362787,0.328304,0.273406,0.246124,0.315344,0.284476,0.237118,0.212644
...,...,...,...,...,...,...,...,...
17975,0.576890,0.639665,0.489684,0.544394,0.535624,0.594491,0.457069,0.508712
17976,0.602951,0.597670,0.498973,0.494448,0.559331,0.554365,0.463186,0.458920
17977,0.398867,0.470063,0.271147,0.320559,0.334062,0.395061,0.220953,0.262280
17978,0.564283,0.675726,0.453359,0.544366,0.517524,0.620505,0.415057,0.499091


---
### Comparison of Applied Metrics
Because the numeric system used for all of these can be different, the best way to compare them is by checking the correlation with the annotator's scores.

In [15]:
# Initialize a dict to be transformed to a df later, for score comparison.
corr_list = ["pearson","kendall"]
scores_dict = {}

for df_name in df_dict.keys():
    for corr in corr_list:
        scores_dict[corr+"_"+df_name] = []
    
scores_index = []

In [16]:
# Thankfully, Pandas has a corr method. Use it on standardized scores obtained previously.

# for each declared corr method, compute the corr between each computed metric and the avg-score column for each considered df.
for df_name in df_dict.keys():
    for corr in corr_list:
        for key in rouge_scores[0].keys():
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(key+ "_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))
        for chrf_score in set(chrf_scores):
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(chrf_score+"_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))
        for bleu_score in bleu_scores_list:
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(key+ "_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))


# Build also a list that will be used to create the index for the scores dataframe.
scores_index.extend(list(rouge_scores[0].keys()))
scores_index.extend(chrf_scores)
scores_index.extend(bleu_scores_list)

In [17]:
scores_df = pd.DataFrame(scores_dict,index=scores_index)
scores_df

,pearson_df_u,kendall_df_u,pearson_df_p,kendall_df_p
rouge-1,0.138382,0.097806,0.143070,0.100834
rouge-2,0.140377,0.099077,0.145481,0.102565
rouge-l,0.140466,0.098409,0.146830,0.103433
chrf_b1_n16,0.357596,0.245061,0.349789,0.239606
chrf_b3_n16,0.340458,0.235044,0.336066,0.231759
chrf_b1_n110,0.324568,0.223361,0.317799,0.218487
chrf_b3_n110,0.354533,0.243727,0.347666,0.238817
chrf_b1_n26,0.330918,0.225935,0.320704,0.218867
chrf_b3_n26,0.324670,0.224175,0.318751,0.219807
chrf_b1_n210,0.333558,0.228636,0.324663,0.222357


In [21]:
for column in scores_df.columns:
    print(f"Max in {column}:")
    print("{}, with {}".format(scores_df[column].argmax(),scores_df[column].max()))


Max in pearson_df_u:
3, with 0.3575959592318776
Max in kendall_df_u:
3, with 0.2450612792866028
Max in pearson_df_p:
3, with 0.34978869406624835
Max in kendall_df_p:
3, with 0.23960622295881834
